In [1]:
# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",
    }
)

# Set the model parameters
param = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particle
    "r_p": 30,  # Positive electrode particle
}

# Set the target number of cycles
target_cycle = 500

print(f"Calculating cycle {target_cycle} ...")

# Define the experiment protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 2C until 2.5 V",
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],
)

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract the LLI loss (in A.h) and calculate the remaining capacity
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600
remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100

print(f"At cycle {target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")


Note: you may need to restart the kernel to use updated packages.
Calculating cycle 500 ...


At t = 151.273 and h = 2.83539e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 151.273 and h = 1.24686e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.97 and h = 4.10172e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.97 and h = 1.74737e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.681 and h = 2.07904e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.681 and h = 2.34404e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.404 and h = 2.42423e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.404 and h = 2.0617e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.136 and h = 2.50451e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 150.136 and h = 3.34076e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At 

At cycle 500, the remaining capacity is: 97.48%
